In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import numpy as np
import io
import itertools
import requests
import random
import shutil
from utils import read_pred_file
import time

In [ ]:
cat ../.env

In [ ]:
API_URL = "http://api:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)


In [ ]:
users = pd.read_csv("../data/csv/API_DATA_DEV - users.csv")
cameras = pd.read_csv("../data/csv/API_DATA_DEV - cameras.csv")
users

Select user via its login

In [ ]:
user = "test77"
organization_id = users[users["login"]==user]["organization_id"].item()

user_cameras = cameras[cameras["organization_id"]==organization_id]
user_cameras

# Send some alerts
Download a file containing relevant images and predictions, then send alerts to selected cameras (with a random azimuth).

In [ ]:
send_alert_from_cam_ids = [2, 4, 5, 7] # select cameras

In [ ]:
if not os.path.isdir("selection-true-positives"):
    print("Images not found, dowloading ...")
    url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/selection-true-positives.zip"
    output_path = "selection-true-positives.zip"

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raises an error for bad status codes

    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    zip_path = "selection-true-positives.zip"
    extract_dir = "selection-true-positives"  # Current directory

    shutil.unpack_archive(zip_path, extract_dir, 'zip')

    print("Extraction completed.")

    

In [ ]:
sequances_folders = glob.glob("selection-true-positives/*")
len(sequances_folders)

In [ ]:
for camera_id in send_alert_from_cam_ids:
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)

    sequances_folder = sequances_folders[camera_id]

    imgs = glob.glob(f"{sequances_folder}/images/*")
    imgs.sort()
    preds = glob.glob(f"{sequances_folder}/labels_predictions/*")
    preds.sort()

    cam_center_azimuth = random.randint(0,360)
    print(f"Sending alerts from camera {camera_id} at azimuth {cam_center_azimuth}")
    for img_file, pred_file in zip(imgs, preds):
    
        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        bboxes = read_pred_file(pred_file)
        response = camera_client.create_detection(stream.getvalue(), cam_center_azimuth, bboxes)
        # Force a KeyError if the request failed
        
        response.json()["id"]

# Create 2 sequences that triangulate
Download a file containning 2 alerts from 2 cameras that triangulate, then it send alerts to the api. 

In [ ]:
if not os.path.isdir("triangulated_sequences"):
    print("Images not found, dowloading ...")
    output_path = "triangulated_sequences.zip"

    url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/"+"triangulated_sequences.zip"

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raises an error for bad status codes

    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    zip_path = output_path
    extract_dir = "triangulated_sequences"  # Current directory

    shutil.unpack_archive(zip_path, extract_dir, 'zip')

    print("Extraction completed.")

cam_triangulation = {
    "12": {
        "azimuth": 226,
        "path": "triangulated_sequences/brison-03"
    },
    "13": {
        "azimuth": 54,
        "path": "triangulated_sequences/serre-de-barre-01"
    }
}

for cam_id, info in cam_triangulation.items():
    camera_token = get_camera_token(API_URL, cam_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)
    info["client"] = camera_client

    seq_folder = info["path"]

    imgs = glob.glob(f"{seq_folder}/images/*")
    imgs.sort()
    preds = glob.glob(f"{seq_folder}/labels_predictions/*")
    preds.sort()

    print(f"Cam {cam_id}: {len(imgs)} images, {len(preds)} preds")  # debug

    info["seq_data_pair"] = list(zip(imgs, preds))

print("Send some entrelaced detections")
for files in itertools.zip_longest(*(info["seq_data_pair"] for info in cam_triangulation.values())):

    for (cam_id, info), pair in zip(cam_triangulation.items(), files):
        if pair is None:
            continue #len of sequences might be different
        img_file, pred_file = pair
        client = info["client"]
        azimuth = info["azimuth"]

        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        with open(pred_file, "r") as file:
            bboxes = file.read()

        response = client.create_detection(stream.getvalue(),azimuth,eval(bboxes) )
        time.sleep(1)

        response.json()["id"]    # Force a KeyError if the request failed
        print(f"detection sent for cam {cam_id}")